In [3]:
import os
os.chdir(r'C:\Users\ydool\Repos\sensor-analysis-assignment')

In [5]:
from extract_features import process_dataset, load_and_process_sample
from visualization import signal_viewer
from imu_pipeline import IMUPipeline
from pathlib import Path
import pandas as pd
import joblib
import os

In [6]:
signal_viewer(
    data_dir=Path('data/raw/train'),
    labels_csv=Path('data/train.csv')
)

# ❓ Questions to Reflect On


- What do you observe when comparing the model’s predictions on the new data to its known performance?

- Is there anything in the data that might explain differences in behavior?

- Can you identify patterns or trends related to when the model succeeds or fails?

- Are there signals or features that seem to affect the model’s reliability?

- What could be done in the short term to handle the current situation?

- What are potential long-term steps to improve model performance in similar scenarios?

- What would you want to explore further if given more time or data?

- What assumptions did the model rely on during training — and are they still valid?

##### Analysis of test data vs. inference data
# note: I used chatGPT to understand what the columns are, as I couldn't find a description of the columns in the task. So if a column is misinterpreted that's a context-availability problem as I am unfamliar with this data. 

The cause I found for the difference in performance is that driver vehicle data somehow got mangled. 
The easiest way to see if when looking at data that should be the same for a vehicle across sessions - device_model, calibration_status, firmware_version, sensor_source, network_type
if you look at cell 162 in the notebook where I filtered for driver_id D1071, and you can see that according to the data, he has 3 vehicles, they have different sensor sources and firmware versions and different calibration status and network types, but in cell 164 when the info for the same driver is filtered, magically there is only 1 row per vehicle.
My intuition says something's wrong with the data pipeline or the way these data points arrive in the pipeline. I don't have access to Source of Truth data so I can't say for sure.

Short term solution is to remove from the pipeline all the session ids from the inference run (they do not overlap with the test data, so we keep the good data).Also if I had access to the pipeline and source of truth data I would remove anything else that could have the same issues.
Long term, this would need to be fixed and monitored. I'd add a validation script that checks for these issues and flags them as they enter the database.

While investigating the 2 datasets, I found these to be true for both:
-firmware version affects model performance (older version - worse results)
-time of day affects model performance (worst in the afternoon, suggesting that the model works well for day time and night when things are either bright or dark, and less so in the inbetween condition)
-vehicle type - SUVs seem to have worse detection rate
-weather - fog decreases performance (which makes sense, it also decreases the human driver's performance because of the limited visibility)

Adding training data for older firmware versions, afternoon rides and SUVs seems like it could improve performance. 

I would like more explanations of the data before exploring it further, but assuming everything chatGPT told me about the columns is correct, given more time I'd explore the model performance relative to different types of vehicle movements, and also I would like to have this data separated into highways and residential streets and dirt roads (and any other road type) because those are very different car motions, indication of the other traffic on the road (other cars have a lot of influence on a vehicle's motion), and also I would like more info on the vehicle - is it electric or gas, size (smaller cars can feel very different from larger cars, or at least for the smaller car I used to have in 2012, it felt a bit wobbly when the winds were high), also speaking of wind weather info could be useful, or maybe just an indication if there are strong winds as this affects a car's movement as well (or at least the care i used to have).

Regarding model assumptions relied on during training - I can't say for sure, it seems to be that the assumptions are along the lines of - everyone has a car or a truck, is driving at morning or at night, with the latest firmware. so diversifying that data would be the key to solving that.